# SQL query from table names - Continued

In [1]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

## The old Prompt

In [7]:
#The old prompt
old_context = [ {'role':'system', 'content':"""
you are a bot to assist in create SQL commands, all your answers should start with \
this is your SQL, and after that an SQL that can do what the user request. \
Your Database is composed by a SQL database with some tables. \
Try to maintain the SQL order simple.
Put the SQL command in white letters with a black background, and just after \
a simple and concise text explaining how it works.
If the user ask for something that can not be solved with an SQL Order \
just answer something nice and simple, maximum 10 words, asking him for something that \
can be solved with SQL.
"""} ]

old_context.append( {'role':'system', 'content':"""
first table:
{
  "tableName": "employees",
  "fields": [
    {
      "nombre": "ID_usr",
      "tipo": "int"
    },
    {
      "nombre": "name",
      "tipo": "varchar"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
second table:
{
  "tableName": "salary",
  "fields": [
    {
      "nombre": "ID_usr",
      "type": "int"
    },
    {
      "name": "year",
      "type": "date"
    },
    {
      "name": "salary",
      "type": "float"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
third table:
{
  "tablename": "studies",
  "fields": [
    {
      "name": "ID",
      "type": "int"
    },
    {
      "name": "ID_usr",
      "type": "int"
    },
    {
      "name": "educational_level",
      "type": "int"
    },
    {
      "name": "Institution",
      "type": "varchar"
    },
    {
      "name": "Years",
      "type": "date"
    }
    {
      "name": "Speciality",
      "type": "varchar"
    }
  ]
}
"""
})

## New Prompt.
We are going to improve it following the instructions of a Paper from the Ohaio University: [How to Prompt LLMs for Text-to-SQL: A Study in Zero-shot, Single-domain, and Cross-domain Settings](https://arxiv.org/abs/2305.11853). I recommend you read that paper.

For each table, we will define the structure using the same syntax as in a SQL create table command, and add the sample rows of the content.

Finally, at the end of the prompt, we'll include some example queries with the SQL that the model should generate. This technique is called Few-Shot Samples, in which we provide the prompt with some examples to assist it in generating the correct SQL.


In [12]:
context = [{'role':'system', 'content':"""
-- Table Definitions
CREATE TABLE employees (
    ID_usr INTEGER PRIMARY KEY,
    name VARCHAR(100)
);

CREATE TABLE salary (
    ID_usr INTEGER,
    year DATE,
    salary FLOAT,
    FOREIGN KEY (ID_usr) REFERENCES employees(ID_usr)
);

CREATE TABLE studies (
    ID INTEGER PRIMARY KEY,
    ID_usr INTEGER,
    educational_level INTEGER,
    Institution VARCHAR(100),
    Years DATE,
    Speciality VARCHAR(100),
    FOREIGN KEY (ID_usr) REFERENCES employees(ID_usr)
);

-- Sample Rows
INSERT INTO employees VALUES (1, 'Alice');
INSERT INTO employees VALUES (2, 'Bob');
INSERT INTO employees VALUES (3, 'Charlie');

INSERT INTO salary VALUES (1, '2023-01-01', 70000);
INSERT INTO salary VALUES (2, '2023-01-01', 85000);
INSERT INTO salary VALUES (3, '2023-01-01', 90000);

INSERT INTO studies VALUES (1, 1, 3, 'Ohio University', '2015-06-01', 'Computer Science');
INSERT INTO studies VALUES (2, 2, 2, 'Ohio State University', '2018-07-01', 'Economics');
INSERT INTO studies VALUES (3, 3, 4, 'MIT', '2012-05-01', 'Physics');

-- Few-Shot Sample Queries

-- Example query 1
-- Get the names of employees earning more than 80000.
SELECT e.name
FROM employees e
JOIN salary s ON e.ID_usr = s.ID_usr
WHERE s.salary > 80000;

-- Example query 2
-- Find all institutions where employees studied.
SELECT DISTINCT Institution
FROM studies;

-- Example query 3
-- List employees with educational_level higher than 2.
SELECT e.name
FROM employees e
JOIN studies st ON e.ID_usr = st.ID_usr
WHERE st.educational_level > 2;
"""}]


In [13]:
# FEW SHOT SAMPLES
context.append({'role':'system', 'content':"""
-- Maintain the SQL order simple and efficient using valid SQLite syntax. Answer the following types of questions clearly and precisely.

-- Example query 4
-- Find the highest salary among employees.
SELECT MAX(salary) AS highest_salary FROM salary;

-- Example query 5
-- List all employee names and their respective specialities.
SELECT e.name, st.Speciality
FROM employees e
JOIN studies st ON e.ID_usr = st.ID_usr;

-- Example query 6
-- How many employees studied at Ohio University?
SELECT COUNT(*) AS num_employees
FROM studies
WHERE Institution = 'Ohio University';
"""
})


In [14]:
#Functio to call the model.
def return_CCRMSQL(user_message, context):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

    newcontext = context.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=newcontext,
            temperature=0,
        )

    return (response.choices[0].message.content)

## NL2SQL Samples
We're going to review some examples generated with the old prompt and others with the new prompt.

In [18]:
#new
context_user = context.copy()
print(return_CCRMSQL("List employees and their salaries.", context_user))

```sql
SELECT e.name, s.salary
FROM employees e
JOIN salary s ON e.ID_usr = s.ID_usr;
```


In [21]:
#old
old_context_user = old_context.copy()
print(return_CCRMSQL("List employees and their salaries.", old_context_user))

This is your SQL:
```sql
SELECT employees.name, salary.salary
FROM employees
JOIN salary ON employees.ID_usr = salary.ID_usr;
```

This SQL query retrieves the names of employees along with their salaries by joining the "employees" table with the "salary" table on the common column "ID_usr".


In [22]:
# new
print(return_CCRMSQL("List institutions and the average salary of their graduates, ordered from highest to lowest, showing only the top 5 results.", context_user))


```sql
SELECT st.Institution, AVG(s.salary) AS avg_salary
FROM studies st
JOIN salary s ON st.ID_usr = s.ID_usr
GROUP BY st.Institution
ORDER BY avg_salary DESC
LIMIT 5;
```


In [23]:
# old
print(return_CCRMSQL("Which institution has graduates with the highest average salary?", old_context_user))


This is your SQL:
```sql
SELECT s.Institution, AVG(sa.salary) AS avg_salary
FROM studies s
JOIN employees e ON s.ID_usr = e.ID_usr
JOIN salary sa ON s.ID_usr = sa.ID_usr
GROUP BY s.Institution
ORDER BY avg_salary DESC
LIMIT 1;
```

This SQL query joins the tables `studies`, `employees`, and `salary` on the user ID to retrieve the institution with the highest average salary among graduates. It calculates the average salary for each institution, orders the results in descending order, and limits the output to the institution with the highest average salary.


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong.
     - What did you learn?

In [25]:
import os
from openai import OpenAI
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

# Function to interact with the OpenAI API
def return_CCRMSQL(user_message, context, model="gpt-3.5-turbo", temperature=0):
    client = OpenAI(api_key=OPENAI_API_KEY)

    new_context = context.copy()
    new_context.append({'role': 'user', 'content': f"question: {user_message}"})

    response = client.chat.completions.create(
        model=model,
        messages=new_context,
        temperature=temperature
    )

    return response.choices[0].message.content

# Define your context with SQL tables related to whales
context = [{'role': 'system', 'content': """
CREATE TABLE whales (
    whale_id INTEGER PRIMARY KEY,
    species VARCHAR(100),
    weight_tons FLOAT,
    length_meters FLOAT,
    habitat VARCHAR(100)
);

INSERT INTO whales VALUES (1, 'Blue Whale', 150, 30, 'Oceanic');
INSERT INTO whales VALUES (2, 'Humpback Whale', 40, 16, 'Coastal');
INSERT INTO whales VALUES (3, 'Beluga Whale', 1.5, 5, 'Arctic');

-- Example Queries
SELECT species FROM whales WHERE weight_tons > 10;
SELECT AVG(length_meters) FROM whales;
"""}]

# Test the function with sample queries
print(return_CCRMSQL("List all whale species and their habitats.", context))
print(return_CCRMSQL("Which whale is the longest?", context))
print(return_CCRMSQL("Find whales living in Coastal habitats.", context))


```sql
SELECT species, habitat FROM whales;
```
To find out which whale is the longest, you can use the following SQL query:

```sql
SELECT species
FROM whales
ORDER BY length_meters DESC
LIMIT 1;
```

This query will retrieve the species of the whale with the longest length in meters from the `whales` table.
```sql
SELECT species 
FROM whales 
WHERE habitat = 'Coastal';
```


### Summary Report: Evaluation of SQL Query Generation with GPT

During our exploration of generating SQL queries with GPT, we experimented with different prompt variations centered around whale-themed tables. The goal was to determine the effectiveness and accuracy of GPT-generated SQL queries based on clearly structured, conversational, and creatively engaging prompts.

### Variations Tested:
1. **Structured and Clear Prompt**
   - Clearly defined tables, fields, and concise example queries.
   - GPT consistently generated accurate and precise SQL queries, effectively interpreting user intent.

2. **Conversational Style Prompt**
   - Prompts were casual and intended to mimic natural conversation.
   - GPT generally responded well, although minor inaccuracies appeared when conversational prompts lacked precision, occasionally leading to vague interpretations.

3. **Creative and Engaging Prompt**
   - Employed thematic storytelling to enhance engagement.
   - GPT's performance was mixed; while some creative prompts produced accurate queries, others led to slight hallucinations or misinterpretations of the intended request due to ambiguity in the storytelling style.

### Variations that Didn't Work Well:
- The creative prompt, while engaging, occasionally caused GPT to hallucinate or misinterpret the intent, resulting in minor inaccuracies. Specifically, GPT was less reliable when queries contained ambiguous or highly creative